# 讀出所有方志階層檔，並存於 Book obj 中

In [1]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC'

In [2]:
import os

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(filepath):
    for file in f:
        if '._' not in file and '.xml' in file and file != 'bookId.xml':
            files.append(os.path.join(r, file))

print(len(files))
print(files[0:5])

3998
['/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97486.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35386.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/92850.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/36842.xml']


In [3]:
import xmltodict, json

Books = {}

def singleChapter(INchapter, INparent, chapterTitles):
    _title = INchapter['title']
    
    _from = ""
    if '@from' in INchapter:
        _from = INchapter['@from']
    
    _to = ""
    if '@to' in INchapter:
        _to = INchapter['@to']
        
    _level = ""
    if '@level' in INchapter:
        _level = INchapter['@level']
        
    _n = ""
    if '@n' in INchapter:
        _n = INchapter['@n']

    # 'parent' = [sub]
#     print(INparent, _title)
    if INparent not in chapterTitles:
        chapterTitles[INparent] = [_title]
    elif _title not in chapterTitles[INparent]:
        chapterTitles[INparent].append(_title)
        
    # [sub] = [parent]
#     if _title not in chapterTitles:
#         chapterTitles[_title] = [INparent]
#     elif INparent not in chapterTitles[_title]:
#         chapterTitles[_title].append(INparent)
    
def section(INarray, INtitle, chapterTitles):
#     print("section::", INtitle, type(INarray), INarray)
    if type(INarray) == dict:
        singleChapter(INarray, INtitle, chapterTitles)
        if 'section' in INarray:
            section(INarray['section'], INarray['title'], chapterTitles) 
        
    elif type(INarray) == list:    
        for ch in INarray:
            section(ch, INtitle, chapterTitles)
            if 'section' in ch:
                section(ch['section'], ch['title'], chapterTitles)
                
    else:
        print(INarray)
        print("ERROR:", type(INarray))
        
    return chapterTitles

# file: [35393].xml
def recursiveSection(INdata):
    chapterTitles = {}
    book = INdata['title']

    if 'section' in INdata:
        titleObj = section(INdata['section'], book, chapterTitles)
    return titleObj

def parseFile(file):
    with open(file) as fp:
        xmlStr = fp.read()
        convertedDict = xmltodict.parse(xmlStr);
        jsonStr = json.dumps(convertedDict, ensure_ascii=False)
        jsonObj = json.loads(jsonStr)

#         print(jsonObj)
        book = jsonObj['book']

        titleObj = recursiveSection(book)
#         print(titleObj)
    
        rstart = file.rfind('/')
        rend = file.rfind('.')
        # file: [35393].xml
        bid = file[rstart+1:rend]
        
        bname = list(titleObj.keys())[0]         
        Books[bid] = {'book': bname, 'data': titleObj}
    
# print(files[0])
# parseFile(files[0])
# print(Books)
# parseFile("/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97725.xml")

In [4]:
# root = AnyNode(id="root")
for file in files:
    parseFile(file)
print(len(Books))
# parseFile(files[0])

3998


In [5]:
# the first 
# print(Books[list(Books.keys())[0]])
# print(Books['17106'])
# print(Books['97725'])
print(Books['35282'])


{'book': '涇縣續志', 'data': {'涇縣續志': ['山澤附（闕）', '節婦（闕）', '涇縣續志序', '涇縣續志纂修姓氏', '涇縣續志目錄', '卷一', '卷二', '卷三', '卷四', '卷五', '卷六', '卷七', '卷八', '卷九'], '卷一': ['學校', '書院', '壇廟', '公館', '橋梁', '津渡', '古蹟', '寺觀'], '書院': ['學田'], '古蹟': ['冡墓'], '卷二': ['食貨', '職官表', '選舉表'], '食貨': ['蠲賑', '恩賚', '户口', '積貯'], '選舉表': ['例仕', '吏仕', '武選舉', '武職', '贈封', '歷襲'], '卷三': ['人物'], '人物': ['名臣', '宦業', '忠節', '孝友', '儒林', '文苑', '武功', '懿行', '尚義', '五世同堂', '百歳', '隱逸', '藝術', '寓賢'], '卷四': ['人物'], '卷五': ['人物'], '卷六': ['人物'], '卷七': ['列女'], '列女': ['已旌', '烈婦：已旌', '貞女', '壽婦', '節婦：已故待旌', '烈婦', '孝女', '現年節婦'], '卷八': ['列女'], '卷九': ['藝文', '災祥', '附刻煙石嶺禁碑并\uedce字']}}


# 哪幾本方志階層不用修正 

In [6]:
import sys

def showLevelGap(level):
    x = ""
    for idx in range(0, level):
        x += "==="
    return x
    
def showSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        prefix = showLevelGap(level)
        print(prefix, s)
        if s not in data:
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)
            
        nxtSubs = data[s]
        showSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def showGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    showSubChapter(firstLevels, data, 1, uppers)
            
# showGazetter('35392')
# showGazetter('17106')
# showGazetter('97725')
# showGazetter('35282')


In [7]:
import sys
import re

def isChineseOnly(word):
    if type(word) != str:
        return False
            
    founds = re.findall(r'[\u4e00-\u9fff]+', word)
    if len(founds) != 1:
#         print("Something not Chinese:", word, founds)
        return False

def checkSubChapter(subs, data, level, uppers, limitLen):
    if level > 10:
        sys.exit()

    for s in subs:        
        # ##FIX##, the s could be dict in 97972. {'hi': '田產附', '#text': '義學'}
        if type(s) != str:
            return False
        
        if isChineseOnly(s) == False:
            return False
        
        if len(s) > limitLen:
            return False
        
        if s not in data:
            continue
            
        if level == 1:
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        if checkSubChapter(nxtSubs, data, level+1, uppers, limitLen) == False:
            return False
    return True

# check gazetter's chapter categories are correct or not
def checkGazetter(bid, limitLen):
    if bid not in Books:
        print("ERROR!!", bid)
        return False
    gazetteer = Books[bid]
    title = gazetteer['book']

    data = gazetteer['data']
    firstLevels = data[title]
    return checkSubChapter(firstLevels, data, 1, [], limitLen)

# checkGazetter('36789', 3)
# checkGazetter('98012', 3)
# checkGazetter('97725', 3)
# checkGazetter('97972', 10)
checkGazetter('97308', 10)


False

In [8]:
# gazetters3 = []
# for b in Books:
#     if checkGazetter(b, 3):
#         gazetters3.append(b)
# print(len(gazetters3))
# print(gazetters3)

In [9]:
# showGazetter('97806')

## Merge the categories

In [10]:
categories = {}

def addCategory(parent, subarr):
    if type(parent) != str:
        return

    if parent not in categories:
        # remove the duplicated
        categories[parent] = list(set(subarr))
    else:
        existList = set(categories[parent])
        subList = set(subarr)
        differList = subList - existList
        categories[parent].extend(list(differList))
        
def mergeSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        if s not in data:
            if level == 1:
                addCategory(s, [])
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        addCategory(s, nxtSubs)
        mergeSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def mergeGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    mergeSubChapter(firstLevels, data, 1, uppers)

In [11]:
# for bid in gazetters3:
#     mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
# categories

In [12]:
# removeList = ['序', '下卷', '上卷', '卷七', '序言', '自叙', '徽縣志']

# remove the elements which should not be the class
def removeElements(ctgrs, removes):
    newCategory = {}
    for key in ctgrs.keys():
        subArr = ctgrs[key]
        if key in removes:
            # remove the class and move the sub classes to upper layer
            for cs in subArr:
                if cs not in removes and cs not in newCategory:
                    newCategory[cs] = []
        else:
            newCategory[key] = subArr
            
    return newCategory


# categs = removeElements(categories, removeList)
# categs

# Check length > 4

In [13]:
# def checkChineseOnly(INctgrs):
#     for key in INctgrs.keys():
#         isChineseOnly(key)
#         subArr = INctgrs[key]
        
#         for elm in subArr:
#             isChineseOnly(elm)
            
# checkChineseOnly(categs)

In [14]:
# gazetters4 = []
# for b in Books:
#     if b not in gazetters3 and checkGazetter(b, 4):
#         gazetters4.append(b)
# print(len(gazetters4))
# print(gazetters4)

In [15]:
# showGazetter('97679')
# for bid in gazetters4:
#     mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
# categories

In [16]:
gazettersN = []
for b in Books:
#     print(b)
    if checkGazetter(b, 30):
        gazettersN.append(b)
print(len(gazettersN))

1549


In [17]:
# showGazetter('97679')
for bid in gazettersN:
    mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
# categories

建寧縣志
弋陽縣志
天長縣志纂輯志稿
蒙陰縣志
無錫志
茶陵縣志
靈川縣志
太康縣志
岳池縣志
續纂光澤縣志
封丘縣志
崇寧縣志
黟縣續志
新城縣志
臨縣志
信邑志稿
無爲縣小志
山東通志
岳陽縣志
德安府志
欒城縣志
潛江縣志稿
陵水縣志
江華縣志
黔陽縣志
建德縣志
太倉州志
懷仁縣志
蓬溪縣志
壽張縣志
洪雅縣續志
江寧縣志
西安縣志
東臺縣志稿
湖廣圖經志書
重修陝西乾州志
邢臺縣志
冀縣志
陝西志輯要
青神縣志
樂亭縣志
鉛山縣志
淶水縣志
建陽縣志
鷄澤縣志
石泉縣志
沁水縣志
懷安縣志
懷安縣志
唐縣志
大寧縣志
續修郟縣志
合肥縣志
寶應圖經
吉安府志
玉山縣志
懷遠縣志
惠安縣志
金山縣志
淳安縣志
會昌縣志
眉州屬志
長寧縣志
筠連縣志
確山縣志
涉縣志
海寧縣志
黔南識略
彭水縣志
寶坻縣志
樂平縣志
湘潭縣志
昭萍志略
武陵縣志
續滕縣志
重修臺灣縣志
襄城縣志
沔縣志
四川總志
臨山衛志
漢陽縣志
續修平利縣志
利津縣續志
小海場新志
翁源縣新志
臨朐續志
灃縣縣志
興文縣志
興寧縣志
平湖縣志
丹棱縣志
行唐縣新志
湖南通志
廉州府志
崇慶縣志
續修臺灣府志
績溪縣志
靜寧州志
永安縣次志
開泰縣志
續修羅江縣志
榆次縣志
安定縣志
鄰水縣志
吉安縣志
續修曲阜縣志
泗虹合志
續興安府志
續修臺灣縣志
臨沂縣志
續修藍田縣志
興濟縣志
烏青鎮志
曲沃縣志
郫縣志
順義縣志
大竹縣志
新登縣志
玉門縣志
江西全省輿圖
新化縣志
臨潼縣志
鄰水縣續志
商南縣志
鄖陽志
沙河縣志
鹽源縣志
鄒縣續志
蒲江縣志
藍山縣志
浦江志略
藍山縣圖志
房縣志鈔
平鄉縣志
新修潼川府志
汝陽縣志
新建縣志
富陽縣輿地小志
營山縣志
靈壽縣志
餘杭縣志稿
華容縣志
續補虞鄉縣新志
懷來縣志
補纂仁壽縣原志
齊東縣志
名山縣新志
梅菉賦志
直隸綿州志
故城縣志
乾州志
重輯楓涇小志
永川縣志
榆次縣志
應州再續志
瀘州直隸州志
宿州志
道州志
回疆志
浯溪新志
嵊縣志
嘉定州志
安東縣志
保安縣志略
烏青文獻
寶山縣志
榮昌縣志
安塞縣志
南和縣志
重修崑山縣志
新會縣志
豐潤縣志
江油縣志
寧鄉縣志
鹿邑縣志
朝城縣志略
長興縣志
德清縣續志
新修成都府志
簡陽縣志
東安縣志
安福縣志
江陰縣志
興寧縣志
禮縣志
鍾祥縣志
欒城縣志
南溪縣志
魚臺縣志
續修

In [22]:
# find possible dups: the longest same strings
def findLongestSameStr(INarr, INlen):
#     OUTarr = {}
    OUTarr = []
    for fidx in range(0, len(INarr)):
        fw = INarr[fidx]
        if len(fw) < INlen:
            continue

        fpf = fw[:INlen]
        for sidx in range(fidx, len(INarr)):
            if fidx == sidx:
                continue

            sw = INarr[sidx]
            if len(sw) < INlen:
                continue

            spf = sw[:INlen]
            if fpf == spf:
                if fpf in OUTarr:
                    if sw not in OUTarr:
#                         OUTarr[fpf].append(sw)
                        OUTarr.append(sw)
                if fpf not in OUTarr:
#                     OUTarr[fpf] = [fw, sw]
                    OUTarr.append(fw)
                    OUTarr.append(sw)
    return OUTarr

def removeLongLengh(INctgrs, INlen):
    doLoop = True
    
    while doLoop:
        removeListN = []
        for prefixLen in range(20, INlen-1, -1):
            if INlen == prefixLen+1:
                doLoop = False
                break

            print(prefixLen)
            # remove keys and assign sub arries to upper layer
            removeListN = findLongestSameStr(list(INctgrs.keys()), prefixLen)
            if len(removeListN) > 0:
                print(removeListN)
                INctgrs = removeElements(INctgrs, removeListN)
                doLoop = True
                break
            elif len(removeListN) == 0:
                doLoop = False
    return INctgrs
            
categories = removeLongLengh(categories, 5)

20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5


In [19]:
# categories

In [23]:
import json

outdir = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/data'
outName = 'processed_json_20190929.json'
outJson = '{0}/{1}'.format(outdir, outName)
with open(outJson, 'w') as fp:
    json.dump(categories, fp, ensure_ascii=False)

In [24]:
import sys
sys.exit()

SystemExit: 